In [19]:
import warnings
import numpy as np
from unidecode import unidecode
import pandas as pd
warnings.filterwarnings('ignore')
class DataProcessor:
    def __init__(self, filename: str) -> None:
        if filename.endswith(".xlsx"):
            self.df = pd.read_excel(filename, skiprows=8, skipfooter=2, header=[0])
        if filename.endswith(".csv"):
            self.df = pd.read_csv(filename, skiprows=8, skipfooter=2, header=[0])
        self.df.replace(np.nan, -1, inplace=True)
        self.df['Mã sinh viên'] = self.df['Mã sinh viên'].astype(int).astype(str)
        
    def get_subject_credit(self):
        df = self.df.drop(columns=['STT', 'Mã sinh viên', 'Họ đệm', 'Tên', 'Học lực'], axis=0)
        df.drop(columns=df.columns[-6:], axis=0, inplace=True)
        return df.iloc[0].to_dict()
    
    def get_all_student_detail(self, class_name):
        first_col = self.df.columns[1]
        last_col = self.df.columns[-7:-2]
        df = self.df[[first_col] + list(last_col)]
        df.columns = df.iloc[0]
        df.drop(index=0, inplace=True)
        df.columns = ['student_id', 'passed_credit', 'score_10', 'score_4', 'score_char', 'rank']
        res = df.to_dict(orient='records')
        
        df = self.df[['Mã sinh viên', 'Họ đệm', 'Tên']].drop(index=0)
        df.columns = ['student_id', 'first_name', 'lastname']
        df['student_name'] = df['first_name'] + " " + df['lastname']
        df['last_name2'] = df.apply(lambda x: unidecode(x['lastname'].lower()), axis=1)
        df['student_gmail'] = df.apply(lambda x: f"{x['lastname']}.{x['student_id']}@iuh.edu.vn", axis=1)
        df.drop(columns=['first_name', 'last_name2'], inplace=True)
        data = df.to_dict(orient='records')
        
        dict1_map = {d['student_id']: d for d in data}
        res = [dict({**d, **dict1_map.get(d['student_id'], {})}) for d in res]
        result_df = pd.DataFrame(res)
        result_df['class_name'] = class_name
        return result_df

In [20]:
processor = DataProcessor("dummy_data/KHDL16A.xlsx")
processor.get_all_student_detail('KHDL15a')

,student_id,passed_credit,score_10,score_4,score_char,rank,lastname,student_name,student_gmail,class_name
0,20011661,45,6.70,2.58,C+,Khá,Ân,Trần Hoà Ân,Ân.20011661@iuh.edu.vn,KHDL15a
1,20010761,35,5.90,2.14,C,Trung bình,Anh,Phạm Tấn Lan Anh,Anh.20010761@iuh.edu.vn,KHDL15a
2,20028461,37,5.50,2.05,C,Trung bình,Bảo,Đinh Dương Bảo,Bảo.20028461@iuh.edu.vn,KHDL15a
3,20083601,38,6.30,2.31,C+,Trung bình,Bảo,Nguyễn Quang Bảo,Bảo.20083601@iuh.edu.vn,KHDL15a
4,20101811,36,5.50,2.10,C,Trung bình,Bảo,Nguyễn Trần Quốc Bảo,Bảo.20101811@iuh.edu.vn,KHDL15a
...,...,...,...,...,...,...,...,...,...,...
58,20098431,42,6.70,2.58,C+,Khá,Trung,Võ Quốc Trung,Trung.20098431@iuh.edu.vn,KHDL15a
59,20022711,42,6.70,2.55,C+,Khá,Trường,Nguyễn Phú Trường,Trường.20022711@iuh.edu.vn,KHDL15a
60,20087481,42,6.90,2.69,C+,Khá,Uyên,Châu Mỹ Uyên,Uyên.20087481@iuh.edu.vn,KHDL15a
61,20105361,36,6.80,2.68,C+,Khá,Viễn,Đặng Chí Viễn,Viễn.20105361@iuh.edu.vn,KHDL15a
